# 1 CNN results 

In [ ]:
import pickle
from src.Utils import *

from src.datasets import load_dataset_by_name

from src.plotting import *
BATCH_SIZE = 64
##############################################################
# Directories, files, parameters
#############################################################


list_datasets = ["caltech_birds2011", "cars196", "cassava", "cats_vs_dogs", "citrus_leaves", "deep_weeds",  "malaria",  "oxford_flowers102", "plant_leaves", "plant_village"]
results_cnns = {}
for dataset_name in list_datasets:
    print("DATASET: " + dataset_name)
    data_loaded = load_dataset_by_name(dataset_name=dataset_name, batch_size=BATCH_SIZE)

    PATH = "results/results_" + dataset_name
    plots_dir = os.path.join(PATH, "plots_" + dataset_name)

    ##############################################################
    # Obtaining best CNN execution (in training set)
    ##############################################################

    file_best_exec_id_vgg19, df_results_cnn_vgg19 = get_best_cnn(path=PATH, cnn_filter="vgg19", plots_dir=plots_dir)
    file_best_exec_id_inceptionresnetv2, df_results_cnn_inceptionresnetv2 = get_best_cnn(path=PATH, cnn_filter="inceptionresnetv2", plots_dir=plots_dir)
    file_best_exec_id_inceptionv3, df_results_cnn_inceptionv3 = get_best_cnn(path=PATH, cnn_filter="inceptionv3", plots_dir=plots_dir)
    file_best_exec_id_densenet201, df_results_cnn_densenet201 = get_best_cnn(path=PATH, cnn_filter="densenet201", plots_dir=plots_dir)
    file_best_exec_id_xceptionv1, df_results_cnn_xceptionv1 = get_best_cnn(path=PATH, cnn_filter="xceptionv1", plots_dir=plots_dir)
    
    ##############################################################
    # General statistics CNN
    ##############################################################

    vgg_stats = generate_cnn_statistics(df_results_cnn=df_results_cnn_vgg19, file_best_exec_id=file_best_exec_id_vgg19, path=PATH, plots_dir=plots_dir, cnn_filter="vgg19", show_tables=False)
    inception_resnetv2_stats = generate_cnn_statistics(df_results_cnn=df_results_cnn_inceptionresnetv2, file_best_exec_id=file_best_exec_id_inceptionresnetv2, path=PATH, plots_dir=plots_dir, cnn_filter="inceptionresnetv2", show_tables=False)
    inceptionv3_stats = generate_cnn_statistics(df_results_cnn=df_results_cnn_inceptionv3, file_best_exec_id=file_best_exec_id_inceptionv3, path=PATH, plots_dir=plots_dir, cnn_filter="inceptionv3", show_tables=False)
    densenet201_stats = generate_cnn_statistics(df_results_cnn=df_results_cnn_densenet201, file_best_exec_id=file_best_exec_id_densenet201, path=PATH, plots_dir=plots_dir, cnn_filter="densenet201", show_tables=False)
    xceptionv1_stats = generate_cnn_statistics(df_results_cnn=df_results_cnn_xceptionv1, file_best_exec_id=file_best_exec_id_xceptionv1, path=PATH, plots_dir=plots_dir, cnn_filter="xceptionv1", show_tables=False)


    results_cnns[dataset_name] = pd.concat([vgg_stats, inception_resnetv2_stats, inceptionv3_stats, densenet201_stats, xceptionv1_stats])
    del results_cnns[dataset_name]["accuracy"]
    results_cnns[dataset_name].columns =  results_cnns[dataset_name].columns.droplevel(0)
#plot_histories_cnn(PATH, plots_dir, file_best_exec_id)


for dname in list_datasets:
    results_cnns[dname].columns = pd.MultiIndex.from_product([[dname], results_cnns[dname].columns])

concat_df = pd.concat(results_cnns.values(), axis=1)
concat_df_test = concat_df.query('set == "Test"')
concat_df_test

concat_df_test.reset_index(level=0, inplace=True)
concat_df_test.reset_index(level=0, inplace=True)
melted = [pd.melt(concat_df_test[[dataset_name, "model", "set"]], id_vars=["model", "set"]) for dataset_name in list_datasets]
pd_melted = pd.concat(melted)
del pd_melted["set"]
pd_melted = pd_melted.reset_index()
pd_melted.set_index("model")
table_cnns = pd_melted.pivot(index=['model', 'variable_1'], columns='variable_0', values='value')
table_cnns = np.round(table_cnns*100, decimals=2)

table_cnns_str = table_cnns.astype(str) + '%'
table_cnns_str.to_latex("paper_tables_plots/table_dataset_info.tex")
table_cnns

In [ ]:
dataset_name = "caltech_birds2011"

PATH = "results/results_" + dataset_name
plots_dir = os.path.join(PATH, "plots_" + dataset_name)
file_best_exec_id_inceptionresnetv2, df_results_cnn_inceptionresnetv2 = get_best_cnn(path=PATH, cnn_filter="inceptionresnetv2", plots_dir=plots_dir)

generate_cnn_statistics(df_results_cnn=df_results_cnn_inceptionresnetv2, file_best_exec_id=file_best_exec_id_inceptionresnetv2, path=PATH, plots_dir=plots_dir, cnn_filter="inceptionresnetv2", show_tables=False)

# 2 Classification of statistical features

In [ ]:
import pickle
import pandas as pd
list_datasets = ["caltech_birds2011", "cars196", "cassava", "cats_vs_dogs", "citrus_leaves", "deep_weeds",  "malaria",  "oxford_flowers102", "plant_leaves", "plant_village"]

statistics_results = {}
for dataset_name in list_datasets:
    PATH = f"results/results_{dataset_name}/2_classification_experiments_ALL_cnn_execs/dict_exp2.pickle" 
    
    with open(PATH, "rb") as handle:
        dict_exp2 = pickle.load(handle)
    
    statistics_results[dataset_name] = dict_exp2[0][["Classifier", "Test_macro_weighted_avg_precision"]]
    statistics_results[dataset_name].columns = ["Classifier", dataset_name]
    statistics_results[dataset_name] = statistics_results[dataset_name].set_index("Classifier")
    
table_stats = pd.concat(statistics_results, axis=1)

table_stats.columns = table_stats.columns.droplevel()
table_stats = np.round(table_stats*100, decimals=2)
table_stats_str = table_stats.astype(str) + '%'
table_stats_str.to_latex("paper_tables_plots/table_stats.tex")
table_stats.style.highlight_max(color = 'green', axis = 0)

# 3 Classification based on CNN probs + best classifier on stat features (Exp 3 avg + exp 4)

### Obtaining average results

In [ ]:
import pickle
import pandas as pd
list_datasets = ["caltech_birds2011", "cars196", "cassava", "cats_vs_dogs", "citrus_leaves", "deep_weeds",  "malaria",  "oxford_flowers102", "plant_leaves", "plant_village"]

avg_results = {}
for dataset_name in list_datasets:
    print(dataset_name)
    PATH = f"results/results_{dataset_name}/2_classification_experiments_ALL_cnn_execs/dict_exp3.pickle" 
    
    with open(PATH, "rb") as handle:
        dict_exp3 = pickle.load(handle)
    
    avg_results[dataset_name] = dict_exp3#[0][["Classifier", "Test_macro_weighted_avg_precision"]]

    for cnn_model in avg_results[dataset_name].keys():

        avg_results[dataset_name][cnn_model] = avg_results[dataset_name][cnn_model].loc["weighted avg"][["precision"]]
        
        
    avg_results[dataset_name] = pd.concat(avg_results[dataset_name], axis=1)
    
    avg_results[dataset_name].columns = [x.split("_")[1] for x in avg_results[dataset_name].columns ]

results_avg = pd.concat(avg_results)

results_avg.index = results_avg.index.droplevel(1)
results_avg["dataset"] = results_avg.index
results_avg = pd.melt(results_avg, id_vars=["dataset"], value_vars=['vgg19', 'inceptionresnetv2', 'inceptionv3', 'densenet201', 'xceptionv1'])
results_avg = results_avg.set_index(['dataset', 'variable'])
results_avg.index = results_avg.index.rename(['Dataset', 'CNN'])
results_avg.columns = ["Avg."]


### Obtaining classification of probs features 


In [ ]:
import pickle
import pandas as pd
import numpy as np
list_datasets = ["caltech_birds2011", "cars196", "cassava", "cats_vs_dogs", "citrus_leaves", "deep_weeds",  "malaria",  "oxford_flowers102", "plant_leaves", "plant_village"]

combination_results = {}
for dataset_name in list_datasets:
    PATH = f"results/results_{dataset_name}/2_classification_experiments_ALL_cnn_execs/dict_exp4.pickle" 
    
    with open(PATH, "rb") as handle:
        dict_exp4 = pickle.load(handle)
    
    combination_results[dataset_name] = dict_exp4
    
    for cnn_model in combination_results[dataset_name].keys():
        
        # Getting average of executions
        averaged_results = pd.concat([x[["Classifier", "Test_macro_weighted_avg_precision"]] for x in combination_results[dataset_name][cnn_model]]).groupby("Classifier").agg({'Test_macro_weighted_avg_precision': ['mean', 'std']})
        
        combination_results[dataset_name][cnn_model] = averaged_results

        
    combination_results[dataset_name] = pd.concat(combination_results[dataset_name])#.reset_index()
    
    combination_results[dataset_name].columns = combination_results[dataset_name].columns.droplevel()
    
results_comb = pd.concat(combination_results).reset_index()

results_comb.columns = ["Dataset", "CNN", "Classifier", "Mean", "Std."]
results_comb["CNN"] = results_comb["CNN"].map(lambda x: x.split("_")[1])

results_comb = results_comb.set_index(['Classifier', 'Dataset','CNN']).unstack('Classifier')

results_comb.columns = results_comb.columns.swaplevel(0, 1)
results_comb.sort_index(axis=1, level=0, inplace=True)

del results_comb[("LogisticRegression", "Std.")]
del results_comb[("KNeighborsClassifier", "Std.")]
del results_comb[("LinearDiscriminantAnalysis", "Std.")]
del results_comb[("SVM-rbf", "Std.")]
del results_comb[("SVM-sigmoid", "Std.")]


results_comb

### Obtaining mean value as CNN base result 

### PRECAUTION! THIS TAKES THE MEAN VALUE, NEXT CELL USES SAME EXECUTION AS THE ONE USED TO TRAIN THE CLASSIFIERS (BEST IN VALID)


In [ ]:
table_cnns_mean = table_cnns[np.in1d(table_cnns.index.get_level_values(1), ['mean', "std"])]

table_cnns_mean = table_cnns_mean.reset_index()
#del table_cnns_mean["variable_0"]
table_cnns_mean = pd.melt(table_cnns_mean, id_vars=["model", "variable_1"])
table_cnns_mean.columns = ['CNN', 'Measure', 'Dataset', 'CNN base']
table_cnns_mean = table_cnns_mean.set_index(['Dataset', 'CNN', "Measure"]).unstack('Measure')


table_cnns_mean[("CNN base mean std", "mean std")] = table_cnns_mean[("CNN base", "mean")].astype(str) + "±" + table_cnns_mean[("CNN base", "std")].astype(str)
del table_cnns_mean[("CNN base", "std")]
table_cnns_mean.columns = table_cnns_mean.columns.droplevel(1)
table_cnns_mean


### Obtaining base CNN results from best execution (validation set)

In [ ]:
from src.Utils import get_best_cnn, get_cnn_predictions_by_model
from src.datasets import load_dataset_by_name
BATCH_SIZE = 64



cnn_results_best_execution = {}
for dataset_name in list_datasets:
    
    data_loaded = load_dataset_by_name(dataset_name=dataset_name, batch_size=BATCH_SIZE)  
    cnn_resuts_path = f"results/results_{dataset_name}"
    
    cnn_results_best_execution[dataset_name] = {}
    
    for cnn_model in ["densenet201", "inceptionresnetv2", "inceptionv3", "vgg19", "xceptionv1"]:
    
        file_best_exec_id_cnn, _ = get_best_cnn(path=cnn_resuts_path, cnn_filter=cnn_model, plots_dir="tmp/")

        report_classes_exp1, cnn_hot_preds_train_labels, cnn_hot_preds_valid_labels, cnn_hot_preds_test_labels, cnn_hot_preds_train, cnn_hot_preds_valid, cnn_hot_preds_test = get_cnn_predictions_by_model(PATH=cnn_resuts_path,
                                 dataset_name=dataset_name,
                                 data_loaded=data_loaded,
                                 BATCH_SIZE=64,
                                 file_best_exec_id=file_best_exec_id_cnn)
    
        cnn_results_best_execution[dataset_name][cnn_model] = report_classes_exp1


for dataset_name in list_datasets:
    for cnn_model in ["densenet201", "inceptionresnetv2", "inceptionv3", "vgg19", "xceptionv1"]:
        cnn_results_best_execution[dataset_name][cnn_model] = pd.DataFrame(cnn_results_best_execution[dataset_name][cnn_model].loc["weighted avg"][["precision"]]).reset_index()
        del cnn_results_best_execution[dataset_name][cnn_model]["index"]
        cnn_results_best_execution[dataset_name][cnn_model]["dataset"] = dataset_name
        cnn_results_best_execution[dataset_name][cnn_model] = cnn_results_best_execution[dataset_name][cnn_model].set_index(['dataset'])

df_best_exec_cnn_base = pd.concat([pd.concat(cnn_results_best_execution[key]) for key in cnn_results_best_execution.keys()])
df_best_exec_cnn_base = df_best_exec_cnn_base.reset_index().set_index(["dataset", "level_0"])
df_best_exec_cnn_base = pd.melt(df_best_exec_cnn_base.reset_index(), id_vars=["dataset", "level_0"])
del df_best_exec_cnn_base["variable"]
df_best_exec_cnn_base.columns = ["Dataset", "CNN", "value"]
df_best_exec_cnn_base = df_best_exec_cnn_base.set_index(['Dataset', 'CNN'])
df_best_exec_cnn_base

### Building table

In [ ]:
# USING AVERAGE FROM CNN EXECUTIONS
all_results_comb = pd.concat([table_cnns_mean, results_avg*100, results_comb*100], axis=1)


# USING BEST CNN BASE RESULT
#all_results_comb = pd.concat([df_best_exec_cnn_base, results_avg, results_comb], axis=1)

table_all = np.round(all_results_comb, decimals=2)
del table_all["CNN base"]

table_all[("RandomForestClassifier", "Mean")] = table_all[("RandomForestClassifier", "Mean")].astype(str) + "±" + table_all[("RandomForestClassifier", "Std.")].astype(str)
del table_all[("RandomForestClassifier", "Std.")]



table_all_str = table_all.astype(str) + '%'
table_all_str.to_latex("paper_tables_plots/table_all.tex")
table_all_str.to_csv("paper_tables_plots/table_all.csv")

def set_color_cell(v, x):
    if v == max(x):
        return "background: green; font-weight: bold" 
    #elif v > x["CNN base"]:
    #    return "background: blue"
    else:
        return ""
    
def set_bold_cell(v, x):
    if v == max(x):
        return "font-weight: bold" 
    else:
        return ""
    
#all_results_comb.style.apply(lambda x: [set_color_cell(v,x) for v in x], axis = 1)#.apply(lambda x: [set_bold_cell(v,x) for v in x], axis = 1)
del all_results_comb["CNN base mean std"]

table_styler = all_results_comb.style.apply(lambda x: [set_color_cell(v,x) for v in x], axis = 1)
table_styler


In [ ]:
##### RUN THIS CELL BEFORE RUNNING ABLATION STUDY IN ORDER TO OBTAIN THE BEST ENSEMBLE FOR EACH DATASET/CNN

all_results_comb.columns = ["CNN base", "Avg.","KNeighborsClassifier", "LinearDiscriminantAnalysis", "LogisticRegression","RandomForestClassifier", "RandomForestClassifierSTD", "SVM-rbf","SVM-sigmoid"]

best_classifiers = all_results_comb[["Avg.", "LogisticRegression","LinearDiscriminantAnalysis","KNeighborsClassifier","SVM-rbf","SVM-sigmoid","RandomForestClassifier"]].idxmax(axis=1).to_dict()
with open("best_ensemble.pickle", "wb") as handle:
    pickle.dump(best_classifiers, handle, protocol=pickle.HIGHEST_PROTOCOL)


# 4 Ablation study

### Extracting avg results for difference calculation

In [ ]:
import pickle
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import numpy as np; np.random.seed(0)
import seaborn as sns; sns.set_theme()

#cnn_model_selected = "inceptionv3"
#cnn_model_selected = "inceptionresnetv2"
#cnn_model_selected = "xceptionv1"
#cnn_model_selected = "densenet201"
cnn_model_selected = "vgg19"


list_datasets = ["caltech_birds2011", "cars196", "cassava", "cats_vs_dogs", "citrus_leaves", "deep_weeds",  "malaria",  "oxford_flowers102", "plant_leaves", "plant_village"]

ablation_study_dict = {}
for dataset_name in list_datasets:
    print(dataset_name)
    
    path = f"ablation_study_stat_out/results_{dataset_name}"
    path_file = [x for x in os.listdir(path) if x.startswith("results_ablation_stat_out") and x.endswith(cnn_model_selected+".pickle")][0]
        
    with open(os.path.join(path, path_file), "rb") as handle:
        dict_dataset = pickle.load(handle)
    
    ablation_study_dict[dataset_name] = dict_dataset
    
    ablation_study_dict[dataset_name] = {key: ablation_study_dict[dataset_name][key] for key in ablation_study_dict[dataset_name].keys()}
    
    
    
    if type(ablation_study_dict[dataset_name][list(ablation_study_dict[dataset_name].keys())[0]]) == list:

        for stat_key in ablation_study_dict[dataset_name].keys():
    
                ablation_study_dict[dataset_name][stat_key] = pd.concat([x[["Classifier", "Test_macro_weighted_avg_precision"]] for x in ablation_study_dict[dataset_name][stat_key]]).groupby("Classifier").mean().reset_index()
    
    for key in ablation_study_dict[dataset_name].keys():
        ablation_study_dict[dataset_name][key].columns = ["Classifier", key]
        ablation_study_dict[dataset_name][key].set_index('Classifier',inplace=True)
    
    ablation_study_dict[dataset_name] = pd.concat(ablation_study_dict[dataset_name].values(),axis=1,sort=False).reset_index()
    ablation_study_dict[dataset_name].rename(columns = {'index':'Classifier'})
    ablation_study_dict[dataset_name] = pd.melt(ablation_study_dict[dataset_name], id_vars=['Classifier'])
    ablation_study_dict[dataset_name]["Dataset"] = dataset_name
    ablation_study_dict[dataset_name] = ablation_study_dict[dataset_name][ablation_study_dict[dataset_name]["Classifier"] == best_classifiers[(dataset_name, cnn_model_selected)]]
    del ablation_study_dict[dataset_name]["Classifier"]
    #ablation_study_dict[dataset_name]["value_diff_mean"] = ablation_study_dict[dataset_name]["value"] - mean_cnns[dataset_name]/100
    ablation_study_dict[dataset_name]["value_diff_max"] = ablation_study_dict[dataset_name]["value"] - df_best_exec_cnn_base.loc[dataset_name, cnn_model_selected][0]

    ablation_study_dict[dataset_name]["value_diff_all_stats"] = ablation_study_dict[dataset_name]["value"] - all_results_comb.loc[dataset_name, cnn_model_selected][best_classifiers[(dataset_name, cnn_model_selected)]]/100
    
        
ablation_study_df = pd.concat(ablation_study_dict.values())

ablation_study_df.columns = ["Stat out", "value", "Dataset", "value_diff_max", "value_diff_all_stats"]

ablation_study_df["norm_value"] = ablation_study_df['value'] / ablation_study_df.groupby('Dataset')['value'].transform('sum')


ablation_study_df_heatmap = ablation_study_df.pivot_table(values='value_diff_all_stats', index='Dataset', columns='Stat out', aggfunc='first')

fig, ax = plt.subplots(figsize=(25, 5))
g = sns.heatmap(ablation_study_df_heatmap, annot=True, center=0.00, cmap=sns.diverging_palette(220, 50, as_cmap=True))
ax.set(xlabel='Statistic left out')
plt.xticks(rotation=15)

ax.set_title(cnn_model_selected,  weight='bold')

plt.savefig(f"ablation_stat_out_heatmap_{cnn_model_selected}.pdf", dpi=1000, bbox_inches='tight')    

In [ ]:
def highlight_negative(cell):
    if type(cell) != str and cell < 0 :
        return 'background: red; color:black' 
    if type(cell) != str and cell > 0.01 :
        return 'background: green; color:black' 
    
ablation_study_df_heatmap.style.applymap(highlight_negative)


